In [136]:
import pandas as pd # dataset load
from pandas import DataFrame as df
import numpy as np

csv = pd.read_csv("C://Users/saepi/Documents/University/grade4/semester2/CapstonDesign2/continue_two_days_rate_20000x.csv",
                 encoding = 'utf-8', index_col = ["person_id"])
# Outlier 제거한 증감률 파일 불러옴

#print(csv.head())

In [137]:
from sklearn.cluster import KMeans
clusters = 6

#kmeans = KMeans(n_clusters = clusters)
kmeans = KMeans(n_clusters = clusters, init='k-means++')
kmeans.fit(csv)

labels = kmeans.labels_

In [138]:
data = kmeans.labels_
csv['kmeans'] = data
ct = pd.crosstab(csv['kmeans'], ['predict'])
#print(ct)

In [139]:
from sklearn.decomposition import PCA

pca = PCA(3)
pca.fit(csv)

pca_data = pd.DataFrame(pca.transform(csv))

#print(pca_data.head())

In [140]:
from matplotlib import colors as mcolors 
import math 
   
''' Generating different colors in ascending order  
                                of their hsv values '''
colors = list(zip(*sorted(( 
                    tuple(mcolors.rgb_to_hsv( 
                          mcolors.to_rgba(color)[:3])), name) 
                     for name, color in dict( 
                            mcolors.BASE_COLORS, **mcolors.CSS4_COLORS 
                                                      ).items())))[1] 
   
   
# number of steps to taken generate n(clusters) colors  
skips = math.floor(len(colors[5 : -5])/clusters) 
cluster_colors = colors[5 : -5 : skips] 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt 
   
fig = plt.figure() 
ax = fig.add_subplot(111, projection = '3d') 
ax.scatter(pca_data[0], pca_data[1], pca_data[2],  
           c = list(map(lambda label : cluster_colors[label], 
                                            kmeans.labels_))) 
   
str_labels = list(map(lambda label:'% s' % label, kmeans.labels_)) 
   
list(map(lambda data1, data2, data3, str_label: 
        ax.text(data1, data2, data3, s = str_label, size = 16.5, 
        zorder = 20, color = 'k'), pca_data[0], pca_data[1], 
        pca_data[2], str_labels)) 
   
plt.show() 

In [ ]:
import seaborn as sns 
  
# generating correlation heatmap 
sns.heatmap(csv.corr(), annot = True) 
  
# posting correlation heatmap to output console  
plt.show() 

In [ ]:
from matplotlib import cm 
  
# generating correlation data 
df = csv.corr() 
df.index = range(0, len(df)) 
df.rename(columns = dict(zip(df.columns, df.index)), inplace = True) 
df = df.astype(object) 
  
''' Generating coordinates with  
corresponding correlation values '''
for i in range(0, len(df)): 
    for j in range(0, len(df)): 
        if i != j: 
            df.iloc[i, j] = (i, j, df.iloc[i, j]) 
        else : 
            df.iloc[i, j] = (i, j, 0) 
  
df_list = [] 
  
# flattening dataframe values 
for sub_list in df.values: 
    df_list.extend(sub_list) 
  
# converting list of tuples into trivariate dataframe 
plot_df = pd.DataFrame(df_list) 
  
fig = plt.figure() 
ax = Axes3D(fig) 
  
# plotting 3D trisurface plot 
ax.plot_trisurf(plot_df[0], plot_df[1], plot_df[2],  
                    cmap = cm.jet, linewidth = 0.2) 
  
plt.show() 

In [129]:
csv.to_csv('kmeans.csv')

In [144]:
db = DBSCAN(eps=0.2, min_samples=5, metric='euclidean')
y_db = db.fit_predict(csv)
#y_db
# DBSCAN으로 해 본 것. 그러나 https://m.blog.naver.com/samsjang/221023672149에 따르면 유클리디안 거리의 경우 차원의 저주 때문에 제대로 된 값이 안 나올 수 있다는 언급.